In [9]:
import os
from tqdm import tqdm

In [2]:
data_dir = 'raw_data'

### XML files

In [5]:
from bs4 import BeautifulSoup
import re

In [32]:
full_set = {}

for file in tqdm(os.listdir(data_dir)):
    if file.endswith('.xml'):
        xml = os.path.join(data_dir, file)

        with open(xml) as f:
            soup = BeautifulSoup(f, 'xml')

        # Clean up enum and itemize tags
        for enum in (soup.find_all('enum') + soup.find_all('itemize')):
            children = enum.findChildren('item', recursive=False)

            # Make sure that all spaces are gone from both sides of the item
            for child in children:
                if child.string:
                    child.string.replace_with(child.string.strip())

            def adjust_ending_char(sent, char):
                if sent[-1].isalpha():
                    # Add punctuation after sentence when the last character is part of a word
                    return sent + char
                elif sent[-1] == '.' or sent[-1] == ';':
                    # Change the punctuation if there's already something there
                    return sent[:-1] + char
                else:
                    # If any other character (such as quotation marks), add punctuation to it
                    return sent + char

            # Make sure that all items end with a punctuation mark
            if len(children) > 1:
                # End all but the last child in ';'
                for child in children[:-1]:
                    if child.string:
                        child.string.replace_with(adjust_ending_char(child.string, ';'))
                
                # End last child in '.'
                if children[-1].string:
                    children[-1].string.replace_with(adjust_ending_char(children[-1].string, '.'))
            else:
                # Only one child: end with '.'
                if children[0].string:
                    children[0].string.replace_with(adjust_ending_char(children[0].string, '.'))
        
        # Get all text_body tags and their stripped contents
        full_body = list(map(lambda x: re.sub(' +', ' ', x.text.strip().replace('\n', ' ')), soup.find_all('text_body')))

        # Make sure there are no "None"s in the body
        full_body = list(filter(lambda x: x != 'None' and bool(x), full_body))

        full_set[file] = full_body


100%|██████████| 79/79 [00:00<00:00, 136.55it/s]


In [33]:
full_set

{'2007 - get real 0.2.xml': [],
 '1998 - themas.xml': [],
 '2008 - keepass.xml': [],
 '0000 - gamma j.xml': ['This is the Software Requirements Specification for GAMMA-J’s Web Store. This Web Store is designed to allow new online store owners a quick and easy means to setup and perform sales and other core business over the internet. This document will outline all of the functions, capabilities and requirements for Version 1 of GAMMA-J’s Web Store. Version 1 is planned for implementation on a “plug and play” USB Key. Future versions will be based on other network appliances.',
  'This document is intended to flush out the requirements by the customer GAMMA-J. The customer can review this document to ensure their needs along with the needs of their user’s are being met in their new Web Store program. The development team will also use this document for guidance on overall design and implementation of the Web Store system. The test and verification team can reference this to ensure the r